# Tutorial 18: Image Classification with CNNs

In this tutorial, we'll build a complete image classification system using Convolutional Neural Networks (CNNs) in BrainState.

## Learning Objectives

By the end of this tutorial, you will be able to:
- Build CNN architectures for image classification
- Load and preprocess image datasets (MNIST-like)
- Implement complete training loops
- Evaluate model performance
- Visualize results and predictions
- Apply data augmentation
- Monitor training progress

## What We'll Build

We'll create:
- A CNN architecture from scratch
- A training pipeline with validation
- Evaluation metrics and visualization
- A complete end-to-end workflow

In [ ]:
import brainstate as bst
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple, Dict, List
import time

# Set random seed for reproducibility
bst.random.seed(42)

print(f"JAX devices: {jax.devices()}")

## 1. Dataset Preparation

We'll create synthetic MNIST-like data for demonstration.

In [ ]:
def generate_synthetic_mnist(n_samples: int = 1000, img_size: int = 28, n_classes: int = 10):
    """Generate synthetic MNIST-like dataset.
    
    Args:
        n_samples: Number of samples to generate
        img_size: Image size (height and width)
        n_classes: Number of classes
        
    Returns:
        images: Array of shape (n_samples, img_size, img_size, 1)
        labels: Array of shape (n_samples,)
    """
    # Generate random images with patterns
    images = bst.random.randn(n_samples, img_size, img_size, 1) * 0.3
    labels = bst.random.randint(0, n_classes, (n_samples,))
    
    # Add class-specific patterns
    for i in range(n_samples):
        label = int(labels[i])
        # Add horizontal lines for even classes
        if label % 2 == 0:
            for j in range(0, img_size, 4):
                images = images.at[i, j:j+2, :, 0].set(
                    images[i, j:j+2, :, 0] + 0.5
                )
        # Add vertical lines for odd classes
        else:
            for j in range(0, img_size, 4):
                images = images.at[i, :, j:j+2, 0].set(
                    images[i, :, j:j+2, 0] + 0.5
                )
    
    # Normalize to [0, 1]
    images = (images - images.min()) / (images.max() - images.min())
    
    return images, labels

# Generate datasets
print("Generating synthetic dataset...")
X_train, y_train = generate_synthetic_mnist(n_samples=2000)
X_val, y_val = generate_synthetic_mnist(n_samples=400)
X_test, y_test = generate_synthetic_mnist(n_samples=400)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")

### Visualize Dataset

In [ ]:
# Visualize sample images
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
axes = axes.flatten()

for i in range(10):
    axes[i].imshow(X_train[i, :, :, 0], cmap='gray')
    axes[i].set_title(f'Label: {y_train[i]}')
    axes[i].axis('off')

plt.suptitle('Sample Images from Training Set')
plt.tight_layout()
plt.show()

## 2. Build CNN Architecture

In [ ]:
class ConvBlock(bst.graph.Node):
    """Convolutional block with Conv -> BatchNorm -> ReLU -> Pool."""
    
    def __init__(self, in_channels, out_channels, kernel_size=3, pool_size=2):
        super().__init__()
        self.conv = bst.nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(kernel_size, kernel_size),
            padding='SAME'
        )
        self.bn = bst.nn.BatchNorm2d(out_channels)
        self.pool_size = pool_size
    
    def __call__(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = jax.nn.relu(x)
        # Manual pooling using jax.lax
        x = jax.lax.reduce_window(
            x, 
            -jnp.inf,
            jax.lax.max,
            (1, self.pool_size, self.pool_size, 1),
            (1, self.pool_size, self.pool_size, 1),
            'VALID'
        )
        return x

class ImageClassifier(bst.graph.Node):
    """CNN for image classification."""
    
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Convolutional layers
        self.conv1 = ConvBlock(in_channels=1, out_channels=32, kernel_size=3)
        self.conv2 = ConvBlock(in_channels=32, out_channels=64, kernel_size=3)
        
        # Fully connected layers
        # After 2 pooling layers: 28 -> 14 -> 7
        self.fc1 = bst.nn.Linear(64 * 7 * 7, 128)
        self.dropout = bst.nn.Dropout(0.5)
        self.fc2 = bst.nn.Linear(128, num_classes)
    
    def __call__(self, x):
        # Convolutional layers
        x = self.conv1(x)  # (batch, 14, 14, 32)
        x = self.conv2(x)  # (batch, 7, 7, 64)
        
        # Flatten
        batch_size = x.shape[0]
        x = x.reshape(batch_size, -1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = jax.nn.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create model
model = ImageClassifier(num_classes=10)

# Test forward pass
test_batch = X_train[:4]
with bst.environ.context(fit=True):
    test_output = model(test_batch)

print(f"Input shape: {test_batch.shape}")
print(f"Output shape: {test_output.shape}")

# Count parameters
total_params = sum(p.value.size for p in model.states(bst.ParamState).values())
print(f"\nTotal parameters: {total_params:,}")

## 3. Training Setup

In [ ]:
def cross_entropy_loss(logits, labels):
    """Compute cross-entropy loss.
    
    Args:
        logits: Model outputs of shape (batch_size, num_classes)
        labels: True labels of shape (batch_size,)
        
    Returns:
        Scalar loss value
    """
    # Convert labels to one-hot
    num_classes = logits.shape[-1]
    one_hot_labels = jax.nn.one_hot(labels, num_classes)
    
    # Compute log softmax
    log_probs = jax.nn.log_softmax(logits, axis=-1)
    
    # Compute loss
    loss = -jnp.mean(jnp.sum(one_hot_labels * log_probs, axis=-1))
    
    return loss

def accuracy(logits, labels):
    """Compute classification accuracy.
    
    Args:
        logits: Model outputs of shape (batch_size, num_classes)
        labels: True labels of shape (batch_size,)
        
    Returns:
        Accuracy as a float
    """
    predictions = jnp.argmax(logits, axis=-1)
    return jnp.mean(predictions == labels)

# Test loss and accuracy
test_loss = cross_entropy_loss(test_output, y_train[:4])
test_acc = accuracy(test_output, y_train[:4])

print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

### Training Step

In [ ]:
def train_step(model, x_batch, y_batch, learning_rate=0.001):
    """Perform one training step.
    
    Args:
        model: The model to train
        x_batch: Input batch
        y_batch: Label batch
        learning_rate: Learning rate
        
    Returns:
        Dictionary with loss and accuracy
    """
    with bst.environ.context(fit=True):
        # Define loss function
        def loss_fn():
            logits = model(x_batch)
            return cross_entropy_loss(logits, y_batch)
        
        # Compute loss and gradients
        loss, grads = bst.augment.grad(
            loss_fn,
            model.states(bst.ParamState),
            return_value=True
        )()
        
        # Update parameters
        for name, grad in grads.items():
            model.states()[name].value -= learning_rate * grad
        
        # Compute accuracy
        logits = model(x_batch)
        acc = accuracy(logits, y_batch)
        
        return {
            'loss': float(loss),
            'accuracy': float(acc)
        }

def eval_step(model, x_batch, y_batch):
    """Perform one evaluation step.
    
    Args:
        model: The model to evaluate
        x_batch: Input batch
        y_batch: Label batch
        
    Returns:
        Dictionary with loss and accuracy
    """
    with bst.environ.context(fit=False):
        logits = model(x_batch)
        loss = cross_entropy_loss(logits, y_batch)
        acc = accuracy(logits, y_batch)
        
        return {
            'loss': float(loss),
            'accuracy': float(acc)
        }

# Test training step
batch_size = 32
x_batch = X_train[:batch_size]
y_batch = y_train[:batch_size]

metrics = train_step(model, x_batch, y_batch, learning_rate=0.001)
print(f"Training step metrics: {metrics}")

## 4. Complete Training Loop

In [ ]:
def create_batches(X, y, batch_size, shuffle=True):
    """Create batches from dataset.
    
    Args:
        X: Input data
        y: Labels
        batch_size: Batch size
        shuffle: Whether to shuffle data
        
    Yields:
        Tuples of (x_batch, y_batch)
    """
    n_samples = X.shape[0]
    indices = np.arange(n_samples)
    
    if shuffle:
        np.random.shuffle(indices)
    
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch_indices = indices[start_idx:end_idx]
        yield X[batch_indices], y[batch_indices]

def train_epoch(model, X_train, y_train, batch_size, learning_rate):
    """Train for one epoch.
    
    Returns:
        Dictionary with average metrics
    """
    losses = []
    accuracies = []
    
    for x_batch, y_batch in create_batches(X_train, y_train, batch_size, shuffle=True):
        metrics = train_step(model, x_batch, y_batch, learning_rate)
        losses.append(metrics['loss'])
        accuracies.append(metrics['accuracy'])
    
    return {
        'loss': np.mean(losses),
        'accuracy': np.mean(accuracies)
    }

def evaluate(model, X, y, batch_size):
    """Evaluate model on dataset.
    
    Returns:
        Dictionary with average metrics
    """
    losses = []
    accuracies = []
    
    for x_batch, y_batch in create_batches(X, y, batch_size, shuffle=False):
        metrics = eval_step(model, x_batch, y_batch)
        losses.append(metrics['loss'])
        accuracies.append(metrics['accuracy'])
    
    return {
        'loss': np.mean(losses),
        'accuracy': np.mean(accuracies)
    }

# Training configuration
config = {
    'num_epochs': 20,
    'batch_size': 64,
    'learning_rate': 0.001,
}

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
}

# Recreate model for fresh training
model = ImageClassifier(num_classes=10)

print("Starting training...")
print("=" * 70)

best_val_acc = 0.0
start_time = time.time()

for epoch in range(config['num_epochs']):
    # Train
    train_metrics = train_epoch(
        model, X_train, y_train, 
        config['batch_size'], 
        config['learning_rate']
    )
    
    # Validate
    val_metrics = evaluate(model, X_val, y_val, config['batch_size'])
    
    # Record history
    history['train_loss'].append(train_metrics['loss'])
    history['train_acc'].append(train_metrics['accuracy'])
    history['val_loss'].append(val_metrics['loss'])
    history['val_acc'].append(val_metrics['accuracy'])
    
    # Track best model
    if val_metrics['accuracy'] > best_val_acc:
        best_val_acc = val_metrics['accuracy']
        best_epoch = epoch
    
    # Print progress
    if epoch % 5 == 0 or epoch == config['num_epochs'] - 1:
        print(f"Epoch {epoch:2d}/{config['num_epochs']}: "
              f"train_loss={train_metrics['loss']:.4f}, "
              f"train_acc={train_metrics['accuracy']:.4f}, "
              f"val_loss={val_metrics['loss']:.4f}, "
              f"val_acc={val_metrics['accuracy']:.4f}")

training_time = time.time() - start_time
print("=" * 70)
print(f"Training completed in {training_time:.2f}s")
print(f"Best validation accuracy: {best_val_acc:.4f} at epoch {best_epoch}")

## 5. Visualize Training Progress

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
epochs = range(len(history['train_loss']))
ax1.plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
ax1.plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy curves
ax2.plot(epochs, history['train_acc'], 'b-', label='Train Accuracy', linewidth=2)
ax2.plot(epochs, history['val_acc'], 'r-', label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Model Evaluation

In [ ]:
# Evaluate on test set
test_metrics = evaluate(model, X_test, y_test, batch_size=64)

print("Test Set Performance:")
print("=" * 50)
print(f"Loss: {test_metrics['loss']:.4f}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"Error Rate: {(1 - test_metrics['accuracy']) * 100:.2f}%")

### Confusion Matrix

In [ ]:
def compute_confusion_matrix(model, X, y, num_classes=10, batch_size=64):
    """Compute confusion matrix.
    
    Returns:
        Confusion matrix of shape (num_classes, num_classes)
    """
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    
    with bst.environ.context(fit=False):
        for x_batch, y_batch in create_batches(X, y, batch_size, shuffle=False):
            logits = model(x_batch)
            predictions = jnp.argmax(logits, axis=-1)
            
            for true_label, pred_label in zip(y_batch, predictions):
                confusion_matrix[int(true_label), int(pred_label)] += 1
    
    return confusion_matrix

# Compute confusion matrix
cm = compute_confusion_matrix(model, X_test, y_test)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm, cmap='Blues', interpolation='nearest')
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Label')
plt.ylabel('True Label')

# Add text annotations
for i in range(10):
    for j in range(10):
        plt.text(j, i, str(cm[i, j]), 
                ha='center', va='center',
                color='white' if cm[i, j] > cm.max() / 2 else 'black')

plt.tight_layout()
plt.show()

# Per-class accuracy
print("\nPer-Class Accuracy:")
print("=" * 50)
for i in range(10):
    class_total = cm[i].sum()
    class_correct = cm[i, i]
    class_acc = class_correct / class_total if class_total > 0 else 0
    print(f"Class {i}: {class_acc:.4f} ({class_correct}/{class_total})")

## 7. Visualize Predictions

In [ ]:
# Get predictions for test samples
n_samples = 20
test_samples = X_test[:n_samples]
test_labels = y_test[:n_samples]

with bst.environ.context(fit=False):
    logits = model(test_samples)
    predictions = jnp.argmax(logits, axis=-1)
    probabilities = jax.nn.softmax(logits, axis=-1)

# Visualize predictions
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i in range(n_samples):
    ax = axes[i]
    
    # Show image
    ax.imshow(test_samples[i, :, :, 0], cmap='gray')
    
    # Title with prediction
    true_label = int(test_labels[i])
    pred_label = int(predictions[i])
    confidence = float(probabilities[i, pred_label])
    
    color = 'green' if true_label == pred_label else 'red'
    ax.set_title(
        f'True: {true_label}, Pred: {pred_label}\nConf: {confidence:.2f}',
        color=color,
        fontsize=10
    )
    ax.axis('off')

plt.suptitle('Model Predictions (Green=Correct, Red=Wrong)', fontsize=14)
plt.tight_layout()
plt.show()

### Prediction Confidence Distribution

In [ ]:
# Analyze prediction confidence
with bst.environ.context(fit=False):
    all_logits = model(X_test)
    all_predictions = jnp.argmax(all_logits, axis=-1)
    all_probs = jax.nn.softmax(all_logits, axis=-1)

# Get max probability for each prediction
max_probs = jnp.max(all_probs, axis=-1)

# Separate correct and incorrect predictions
correct_mask = (all_predictions == y_test)
correct_probs = max_probs[correct_mask]
incorrect_probs = max_probs[~correct_mask]

# Plot confidence distributions
plt.figure(figsize=(10, 5))

plt.hist(correct_probs, bins=20, alpha=0.7, label='Correct Predictions', color='green')
plt.hist(incorrect_probs, bins=20, alpha=0.7, label='Incorrect Predictions', color='red')

plt.xlabel('Prediction Confidence')
plt.ylabel('Count')
plt.title('Distribution of Prediction Confidence')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average confidence for correct predictions: {jnp.mean(correct_probs):.4f}")
print(f"Average confidence for incorrect predictions: {jnp.mean(incorrect_probs):.4f}")

## 8. Feature Visualization

In [ ]:
# Visualize learned filters
def visualize_conv_filters(model, layer_name='conv1'):
    """Visualize convolutional filters."""
    # Get first conv layer
    conv_layer = getattr(model, layer_name)
    filters = conv_layer.conv.weight.value  # Shape: (out_ch, in_ch, kh, kw)
    
    n_filters = min(16, filters.shape[0])
    
    fig, axes = plt.subplots(4, 4, figsize=(10, 10))
    axes = axes.flatten()
    
    for i in range(n_filters):
        # Get filter for first input channel
        filter_img = filters[i, 0, :, :]
        
        axes[i].imshow(filter_img, cmap='gray')
        axes[i].set_title(f'Filter {i}')
        axes[i].axis('off')
    
    plt.suptitle(f'Learned Filters in {layer_name}')
    plt.tight_layout()
    plt.show()

visualize_conv_filters(model, 'conv1')

## Summary

In this tutorial, we built a complete image classification system:

1. **Dataset Preparation**: Generated synthetic MNIST-like data
2. **CNN Architecture**: Built a CNN with conv blocks and FC layers
3. **Training Setup**: Implemented loss, accuracy, and training steps
4. **Training Loop**: Complete training with validation
5. **Visualization**: Training curves and learning progress
6. **Evaluation**: Test set performance and confusion matrix
7. **Predictions**: Visualized model predictions and confidence
8. **Feature Visualization**: Examined learned convolutional filters

## Key Takeaways

- **CNNs** are powerful for image classification
- **Training loops** require careful batch handling
- **Validation** is crucial for monitoring overfitting
- **Visualization** helps understand model behavior
- **Evaluation metrics** beyond accuracy provide insights
- BrainState makes it easy to build production-ready models

## Next Steps

In the next tutorial, we'll explore:
- **Sequence Modeling**: RNNs, LSTMs, and Transformers
- Text generation and language models
- Attention mechanisms
- Advanced sequence architectures